# Extracting Data using Pydantic

In [36]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAI,ChatOpenAI
from typing import Optional,Literal, List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [67]:
class Product(BaseModel):
    name: str = Field( description="Holds the name of the product")
    type: str=Field(description="Holds type of the product like Electronics, Home Appliances, Beauty ")
    sentiment: Literal['neutral', 'positive', 'negative']= Field(..., description="Sentiments of the product review")

In [63]:
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',
         'you are an expert for data extraction.'
         'you will be provided a text from which you extract relevant information.'
         'if you dont know the value of an attribute asked to extract, return numm for that attribute value'),
        ('human', '{text}')
        
    ]
)

In [57]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [68]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [69]:
review="The laptop battery sucks, and i have to keep it on charging always"

In [70]:
chain= prompt | llm.with_structured_output(schema=Product)
chain.invoke({"text": review})

Product(name='laptop', type='Electronics', sentiment='negative')

In [71]:
class Products(BaseModel):
   all_products: List[Product] #= Field(..., description="List of extracted products")
    

In [72]:
chain = prompt | llm.with_structured_output(schema=Products)
review='I bought two mobiles in the past. The Apple iphone battery is awsome. The smasung note i2 battery decrease life in one year only'
chain.invoke ({"text":review})

Products(all_products=[Product(name='Apple iphone', type='Electronics', sentiment='positive'), Product(name='samsung note i2', type='Electronics', sentiment='negative')])